In [13]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

#Makes some extra dataframes
df = purchase_data.copy()
df2 = df.copy()

In [14]:
#Checkin out the data 
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [15]:
# Gets the total number of players
players = pd.unique(df["SN"])
players_df = pd.DataFrame(players)
p_count = players_df.count()

#Makes a dataframe to hold the total number of players
total_p = pd.DataFrame()
total_p["Total Players"] = p_count
total_p.head()


,Total Players
0,576


In [16]:
#Gets the total number of unique items
items = pd.unique(df["Item ID"])
items_df = pd.DataFrame(items)
i_count = items_df.count()

#Gets the total revenue 
revenue = df["Price"].sum()

#Calculates the Number of Purchases
purchases = df["Purchase ID"].count()

#Calculates the average price per sale
average_price = revenue/purchases

#Makes a dataframe to hold the incredibly valuable information
summary = pd.DataFrame()
summary["Number of Unique Items"] = i_count
summary["Average Price"] = average_price.round(2)
summary["Number of Purchases"] = purchases
summary["Total Revenue"] = revenue
summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.05,780,2379.77


In [17]:
#Creates a Dataframe with only unique players and their genders
unique = df2.drop(columns = ["SN","Item ID","Item Name","Price","Purchase ID"])
unique = unique.merge(players_df, how='outer', left_index = True, right_index = True)
unique = unique.dropna()

#Takes the value count of each player by gender and assigns it to a variable
gender = unique["Gender"].value_counts()
male = gender[0]
female = gender[1]
na = gender[2]
gsum = male + female + na
percent_m = (male/gsum)*100
percent_f = (female/gsum)*100
percent_n = (na/gsum)*100

#Creates a new DataFrame with all the information
gender_df = pd.DataFrame(
    {"Gender": ["Male", "Female", "Other / Non-Disclosed"],
     "Total Count": [male,female,na],
     "Percentage of Players" : [percent_m.round(2),percent_f.round(2),percent_n.round(2)]  
    }
)
gender_df = gender_df.set_index("Gender")
gender_df

,Total Count,Percentage of Players
Gender,,
Male,476,82.64
Female,88,15.28
Other / Non-Disclosed,12,2.08


In [18]:
#The code before this would have been a lot shorter if I knew the difference between set_index and groupby
#luckily I figured out groupby so get ready for much neater code

#Total purchases by gender
purchases_gen = df.groupby("Gender")["Purchase ID"]
test = purchases_gen.count()

#Mean price per item
mean_gen = df.groupby("Gender")["Price"].mean()
#mean_gen

#Total profit by gender
total_gen = df.groupby("Gender")["Price"].sum()
#total_gen

#FIXXXXXXXXXXXXXXX THISSSSSSSSSSSSSSSSS LATERRRRRRRRRRRRRRRRRRRRRRRRRRRR
#avg_gen = df.groupby("Gender")["Price"].mean().sum()
#avg_gen

#Creates a Dataframe with all the wonderful
purchases = pd.DataFrame(
    {#"Gender": ["Male", "Female", "Other / Non-Disclosed"],
     "Purchase Count": test,
     "Average Purchase Price" : mean_gen.round(2),
     "Total Purchase Value" : total_gen
    }
)
purchases

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,3.20,361.94
Male,652,3.02,1967.64
Other / Non-Disclosed,15,3.35,50.19


In [27]:
#Makes bins to sort ages
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]
ages = ["<10", "10-14","15-19", "20-24", "25-29", "30-34", "35-39", ">=40"]

#Creates the binned information and puts it into a new dataframe
ages_cut = pd.cut(unique["Age"], bins, labels=ages)
ages_df = pd.DataFrame()
ages_df["Total Count"] = ages_cut.value_counts()
ages_df["Percentage of Players"] =  ((ages_cut.value_counts() / 576 ) * 100).round(2) #for some reason diving my p_count breaks it
ages_df = ages_df.sort_index()
ages_df

,Total Count,Percentage of Players
<10,23,3.99
10-14,39,6.77
15-19,145,25.17
20-24,247,42.88
25-29,54,9.38
30-34,42,7.29
35-39,22,3.82
>=40,4,0.69


In [43]:
#Creates a dataframe with all the beautiful data
ages_stats = df
ages_stats["Age Range"] = pd.cut(df["Age"], bins, labels= ages)

#More fun with groupby
#Finds The total amount of sales per age range
count_age = ages_stats.groupby("Age Range")["Price"]
count_age = count_age.count()

#Finds the mean price per sale
price_avg_age = ages_stats.groupby("Age Range")["Price"].mean() 

#Finds the total profit of all sales sorted by age groups
price_total_age = ages_stats.groupby("Age Range")["Price"].sum()

#FIXXXXXXXXXXXXXXX THISSSSSSSSSSSSSSSSS LATERRRRRRRRRRRRRRRRRRRRRRRRRRRR
#avg_age_pers = df.groupby("Gender")["Price"].mean().sum()
#avg_age_pers

#Creates a dataframe with all this wonderful information
age_summary_stuff = pd.DataFrame({
    "Purchase Count": count_age,
    "Average Purchase Price": price_avg_age.round(2),
    "Total Purchase Value": price_total_age
})
age_summary_stuff

,Purchase Count,Average Purchase Price,Total Purchase Value
Age Range,,,
<10,32,3.40,108.96
10-14,54,2.90,156.60
15-19,200,3.11,621.56
20-24,325,3.02,981.64
25-29,77,2.88,221.42
30-34,52,2.99,155.71
35-39,33,3.40,112.35
>=40,7,3.08,21.53
